In [1]:
import MDAnalysis as mda
import numpy as np
import os
import shutil
import csv
from utils import *
import warnings 

# Suppress warnings specific to MDAnalysis
warnings.filterwarnings("ignore", category=UserWarning, module="MDAnalysis")

c:\Users\gbonn\anaconda3\envs\md-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# load receptor universe and extract the different parts of the protein int1 receptor 
head_dir = '/projects/p30041/gbf4422/5EJ5/TEST_WORKFLOW/int1/'

# dictionary of charged protein residues with key being the Amber resname and the value being the net charge of that residue
residue_charge_dict = {'ARG':1,'LYS':1,'HIP':1,'ASP':-1,'GLU':-1,'MG':2}

# specify active atoms
by_dist = True 
QM_sphere_r = 2 # Angstroms (a value of 2 will give only the ThDP intermediate (auto adds MG))
active_sphere_r = 8
    
# TODO calculate protein charge automatically (or read it in from leap.log file)
base_charge = -4 # INI without accounting for R groups    
intermediate = 'INI'

all_substrates = ['3']
for curr_substrate in all_substrates:
    print('Current Substrate',curr_substrate)
    curr_dir = head_dir + curr_substrate + '/'
    output_dir = '/projects/p30041/gbf4422/sensitivity_testing/rotation_test/'

    complex = mda.Universe(curr_dir+'MD/run_equil_fail/aligned_last_frame.pdb')

    # get all atoms near the protein excluding NaCl
    trimmed_complex_initial = complex.select_atoms("(protein or resname INI or resname MG) or ((around 2.5 protein) and resname WAT)").residues
    # Renumber residues manually
    for i, residue in enumerate(trimmed_complex_initial):
        residue.resid = i + 1  # Assign new resid starting from 1
    
    write_universe(output_dir  + 'prep/','initial.pdb',trimmed_complex_initial)
    edit_protein_files(output_dir  + 'prep/','initial.pdb')

    trimmed_complex = mda.Universe(output_dir  + 'prep/initial.pdb')
    
    if curr_substrate in ['4','6','11','16']:
        additional_charge = -1
    else:
        additional_charge = 0
   
    residue_charge_dict[intermediate] = base_charge + additional_charge
    MM_charge = 0
    for residue in trimmed_complex.residues:
        curr_resname = residue.resname
        if curr_resname in residue_charge_dict:
            MM_charge += residue_charge_dict[curr_resname]

    # get the QM atoms and residues (automatically includes Mg2+)
    # get INI atoms 
    ini = trimmed_complex.select_atoms("resname " + intermediate)
    # get the carbonyl carbon of the substrate
    #atom_dict = get_substrate_aka_indexes(ini)
    atom_dict = get_ini_indexes(ini)
    C2_index = atom_dict['C2']
    C2_id = ini.atoms[C2_index].index
    C2_atom = trimmed_complex.select_atoms("index " +  str(C2_id))
    QM_residues, active_residues, QM_atoms_indexes,active_atoms_indexes,fixed_atoms_indexes = get_atoms_by_distance(trimmed_complex,QM_sphere_r,active_sphere_r,C2_id)

    QM_residues_resids = [residue.resid for residue in QM_residues]
    active_residues_resids = [residue.resid for residue in active_residues]
    
    # simplify lists to write to file
    QM_list = simplify_integer_list(QM_atoms_indexes)
    active_list = simplify_integer_list(active_atoms_indexes)
    # calculate the charge of our system
    total_QM_charge = 0
    # get charge of QM region 
    for residue in QM_residues:
        resname = residue.resname
        if resname in residue_charge_dict:
            total_QM_charge += residue_charge_dict[resname]

    #write_MM_orca_script(active_list,MM_charge,output_dir+'MM_Opt_Active_10/')
    shutil.copy(curr_dir + 'MD/prep/INI.mol2', output_dir  + 'prep/INI.mol2')
    shutil.copy(curr_dir + 'MD/prep/INI.frcmod', output_dir  + 'prep/INI.frcmod')


    write_QMMM_orca_script(QM_list,active_list,total_QM_charge,output_dir  + 'run/')
    shutil.copy('scripts/template_orca_job_expanse.sh',output_dir  + 'run/' + 'orca_job_expanse.sh')
    write_resids_to_csv(output_dir  + 'run/',f'QM_{QM_sphere_r}_and_{active_sphere_r}A_Active_residues.csv',QM_residues_resids,active_residues_resids)